# 0 Imports and helper functions

https://gallery.azure.ai/Experiment/837e2095ce784f1ba5ac623a60232027

In [0]:
import sklearn
import pandas as pd
from sklearn import tree
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import time
from sklearn.metrics import roc_curve, auc
import json

%matplotlib inline

# 1 Loading the data

In [0]:
# from google.colab import drive
# drive.mount('/content/drive')


In [0]:
# delays = pd.read_csv("/content/drive/My Drive/xylosai/flightDelay/FlightDelaysData.csv",header=0)

In [0]:
# weather = pd.read_csv("/content/drive/My Drive/xylosai/flightDelay/WeatherDataset.csv",header=0)

In [0]:

delays = pd.read_csv("/content/drive/My Drive/xylosai/flightDelay/FlightDelaysData.csv",header=0)
weather = pd.read_csv("/content/drive/My Drive/xylosai/flightDelay/WeatherDataset.csv",header=0)

In [0]:
delays.head()

In [0]:
delays.dtypes

# 2 Cleaning weather data

## 2.1 Exploring the weather data

Pandas provides some native exploration methods that can be called on a dataframe. we will use describe() and plot(). 

 1. [describe()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.describe.html) provides basic statistics for every column. The output is another dataframe.
 2. [plot()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.plot.html) is used to plot column data. By default it creates a line plot, but different plots are possible (see docs). It only plots numerical data. For visualizing categorical data, we will use value_counts() on a column first (see below).

In [0]:
weather.head()

Display a list of all columns with their dtypes.

In [0]:
weather.dtypes

In [0]:
weather.describe()

From the table above, we conclude that all the data is from the **Year** 2013. There are **406 516 records** in total. (you can also check this with *len(weather)*)

## 2.1 Dropping columns with lots of empty strings

Print the unique values of  **ValueForWindCharacter**. Notice the empty string.

In [0]:
weather["ValueForWindCharacter"].unique()

How many (as a percentage) of the ValueForWindCharacter values are empty strings?

In [0]:
perc = float(weather[weather["ValueForWindCharacter"] == ' ']["ValueForWindCharacter"].count())/len(weather)
print(perc)

Let's do the same for **PressureTendency, WeatherType, PressureChange** and **HourlyPrecip**

In [0]:
weather["PressureTendency"].unique()

In [0]:
perc = float(weather[weather["PressureTendency"] == ' ']["PressureTendency"].count())/len(weather)
print(perc)

In [0]:
weather["WeatherType"].unique()

In [0]:
perc = float(weather[weather["WeatherType"] == ' ']["WeatherType"].count())/len(weather)
print(perc)

In [0]:
weather["PressureChange"].unique()

In [0]:
perc = float(weather[weather["PressureChange"] == ' ']["PressureChange"].count())/len(weather)
print(perc)

In [0]:
weather["HourlyPrecip"].unique()

In [0]:
perc = float(weather[weather["HourlyPrecip"] == ' ']["HourlyPrecip"].count())/len(weather)
print(perc)

All the columns identified above, contain mostly empty strings. We will drop these columns.

In [0]:
weather.drop(columns=["ValueForWindCharacter","PressureTendency","WeatherType","PressureChange","HourlyPrecip"],inplace=True)
weather.head()

## 2.2 Exploring some other columns and converting to one-hot



**Airport ID** is a categorical value. How many different Airports are in the dataset? How will we deal with categorical values later?

In [0]:
print("unique airports: ",weather["AirportID"].unique().shape[0])
weather["AirportID"].unique()

We will not one-hot-encode airportID yet, we will first need it later to join the weather data with the flight data.

How many unique values does the **Skycondition** column have? Is it categorical or numerical?



In [0]:
print("unique SkyCondition: ",weather["SkyCondition"].unique().shape[0])
weather["SkyCondition"].unique()

It contains condition codes that we don't understand.  Since there are so many different values, one-hot-encoding would be impractical. We will drop this column.

In [0]:
weather.drop(columns=["SkyCondition"],inplace=True)

**RecordType** has only four unique values, but their meaning is unknown. We will assume they are not very meaningful and we will drop this column.

In [0]:
weather["RecordType"].unique()

In [0]:
weather.drop(columns=["RecordType"],inplace=True)

 **Intermediate result**

In [0]:
weather.head()

In [0]:
weather.dtypes

## 2.3 Removing the M's and casting to numerical. One-hot-encoding WindDirection

A lot of numerical values are parsed as an object (see dtypes list above) because they contain some weird "M" values.
Let's count the M in some of these columns

From printing the dtypes, **Visibility** is an 'object' type. This suggests that it contains string values. Let's explore this column.

In [0]:
weather["Visibility"].unique()

Note the following 'unique' values:

-  '10.00' (string)
- 10.0 (float)
- ' 10.00' (string, but with leading white space)

Most values are numerical, but it also contains the 'M' value. Since plot() only takes numerical data, we will make a bar chart of this data by first calling value_counts() on the Series. This returns a Dataframe with the number of matching values for each unique visibility value. Optionally, try calling value_counts() without plot() to see what it does.



In [0]:
weather["Visibility"].value_counts().plot(kind='bar',figsize=(10,10))

Most values of visibility are 10.0, the 'M' value rarely appears.

We also notice the 'M' value in other columns. Let's zoom in to 3 more columns as an example: **DryBulbFarenheit, Altimeter, SealevelPressure**. Let's calculate the percentage of M's in these columns.

In [0]:
for column in ["Visibility","DryBulbFarenheit", "Altimeter", "SeaLevelPressure"]:
  perc = float(weather[weather[column] == 'M'][column].count())/len(weather)
  print("Fraction of M's in column {0}: {1}".format(column,perc))

We will remove the column SeaLevelPressure since it contains a lot of useless M's. 

In [0]:
weather.drop(columns=["SeaLevelPressure"],inplace=True)

In the other columns, the M value is rarely seen. We would prefer to keep these columns, since we are not sure a priori that these columns have no influence on the flight delay.

We decide to further clean our data by keeping the columns but simply removing all rows from the dataframe that has an M somewhere.

The function below removes all rows with an M for a general dataframe, from all columns with dtype *object*. Afterwards it attempts to cast the column to float.


In [0]:
def remove_M(df):
    original_length = len(df)
    for c in df.columns.tolist():
        if df.dtypes[c] == 'object':
            m = df[df[c] == "M"][c].count()
            print("Removed {0} in column {1}".format(m,c))
            df = df[df[c] != "M"]
            try: 
                df[c] = df[c].astype(np.float64)
            except:
                pass
    n_removed = original_length - len(df)
    print("")
    print("Removed {0} of {1}".format(n_removed,original_length))
    return df

In [0]:
weather = remove_M(weather)

In [0]:
weather.dtypes

Apparently,  columns **WindSpeed** and **WindDirection** still have weird strings as value, since we were not able to cast them to floats. 

In [0]:
print("unique WindSpeed: ",weather["WindSpeed"].unique().shape[0])
weather["WindSpeed"].unique()

Windspeed has empty strings as value, but only very few. We simply remove the rows with an empty WindSpeed from the dataframe, then we can cast the column to numerical.

In [0]:
weather[weather["WindSpeed"] == '  ']["WindSpeed"].count()

In [0]:
weather = weather[weather["WindSpeed"] != '  ']
weather["WindSpeed"] = weather["WindSpeed"].astype(np.int64)

**Winddirection** has some value 'VR' that appears an amount of times that is not negligible.

In [0]:
print("unique WindDirection: ",weather["WindDirection"].unique().shape[0])
weather["WindDirection"].unique()

In [0]:
weather[weather["WindDirection"] == 'VR ']["WindSpeed"].count()

The Winddirection ranges from 0 to 360 degrees. We will use one-hot-encoding for the WindDirection, by splitting it up into categories. Every category represents a range of 40 degrees. Another category is reserved for the 'VR' case.

Review the code in the cell below. This cell performs this one-hot-encoding and splits the column into seperate columns for each one-hot-encoded case. 

In [0]:
direction_categories = [0,40,80,120,160,200,240,280,320]
def check_cat(x,cat):
    try: #try-catch block because VR cannot be compared to int.
        if int(x)>cat-20 and int(x)<=cat+20:
            return 1
        else:
            return 0
    except:
        return 0
    
weather["dir_VR"] = weather["WindDirection"].apply(lambda x: 1 if x == 'VR ' else 0)
for cat in direction_categories:
    weather["dir_{0}".format(cat)] = weather["WindDirection"].apply(lambda x: check_cat(x,cat))
weather.drop(columns=["WindDirection"],inplace=True)

The resulting dataframe has columns "dir_0" up to "dir_320"

In [0]:
weather.head()

In [0]:
weather.dtypes

## 2.4 Handling time

We will use the time information to join the weather dataframe with the delay dataframe. For this, we must format the time information for every row into a fixed format in one column.

We round time to the nearest hour. Time of weather data is rounded UP and time of flight data will be rounded DOWN, to make sure that weather conditions after the flight departure/arrival are not a deciding factor. We create a **datetime** column with strings in the format "YYYY-M-D-H". 

What about the timezone? The weather data is reported in local time. We will leave this in local time and use this to join it to the delay data, where arrival and departure times are also in local times.

In the original dataframe, time of day is written in the HH-mm format:

In [0]:
weather['Time'].head(5)

In [0]:
# function to round the time of day (in HH-mm format) to the nearest hour.
def hhmm_to_h(hhmm, mode):
    #mode = 'up' or 'down'
    if not (mode == 'up' or mode == 'down'):
        raise Exception("mode must be up or down")
    if len(str(hhmm)) <= 2:
        if mode == 'up':
            h = 1
        else:
            h = 0
    else:
        h = int(str(hhmm)[:-2])
        if mode == 'up':
            h += 1
    return h

In [0]:
#first create a column "hour" to round the time to the nearest hour.

weather["hour"] = weather["Time"].apply(lambda x: hhmm_to_h(x,mode='up'))

In [0]:
weather['hour'].head(5)

In [0]:
#then, create a datetime string from the columns Year, Month, Day and hour. Note how operations on series can be easily done thanks to broadcasting.

weather["datetime"] = weather["Year"].astype(str)+'-'+weather["Month"].astype(str)+'-'+weather["Day"].astype(str)+'-'+weather["hour"].astype(str)

In [0]:
weather['datetime'].head(5)

In [0]:
weather.dtypes

## 2.5 Result

Dropping some columns that we no longer need.

In [0]:
weather.drop(columns=["Year","Month","Day","Time","TimeZone","hour"],inplace=True)

The temperature information is redundant: it has both the farenheit and celcius values. We will drop the Farenheit columns. We also drop the Altimeter column.

In [0]:
weather.drop(columns=["DryBulbFarenheit","WetBulbFarenheit","DewPointFarenheit","Altimeter"],inplace=True)

In [0]:
weather.head()

## 2.6 memory usage and dtypes

When working with small amounts of data, Pandas works without any problems. For larger datasets, memory usage of the Pandas dataframes becomes to high and the runtime could possible crash. Therefore, we will try to optimize memory usage. If we don't, this notebook will crash when running in the Colaboratory environment. 

In order to minimize the memory footprint, we have to look at the dtypes of the columns.

In [0]:
weather.dtypes

Pandas uses Numpy arrays internally. Columns of the same dtype are internally stored together in one Numpy Array. 

The choice of dtype determines the memory usage of the column. A more detailed discussion about memory usage in Pandas is found in this [blog](https://www.dataquest.io/blog/pandas-big-data/), but the ongoing disucssion will make sense without reading the blog.

We can inspect a dataframes memory usage as follows:

In [0]:
weather.info(memory_usage='deep',max_cols=1) #without memory_usage = 'deep', the displayed memory usage is a less accurate approximation.

**currently, our memory usage of the weather dataframe is about 80MB**

In [0]:
weather.head()

Observe the possible values for the following columns...

In [0]:
weather["WindSpeed"].unique()

In [0]:
weather["RelativeHumidity"].unique()

In [0]:
weather["Visibility"].unique()

In [0]:
#weather["Altimeter"].unique()

In [0]:
weather[["StationPressure","Visibility","DryBulbCelsius","WetBulbCelsius","DewPointCelsius","RelativeHumidity"]].head()

In [0]:
print("minimum and maximum column values")
for columname in ["Visibility","StationPressure"]:
  print("{0}: minimum = {1}, maximum = {2}".format(columname,weather[columname].min(),weather[columname].max()))

The interger type has subtypes int8, int 16, int32 and int64. These subtypes have a different memory footprint. An int8 is stored in one byte (8 bits), so it can take 2^8=256 values in total. Thus, it can represent numbers between -128 and 127 (including 0). An int64 is stored in 8 bytes and can take much larger values.

**(1) integers**

Currently, all our integer columns are int64. We can reduce our memory usage a lot. One of our integer columns, windspeed, ranges from 0 to 64. There is clearly no need for an int64, an int8 would be sufficient. Even the one-hot encoded columns are int64, while they only take the values 0 and 1. 

A handy way to check the minimal and maximum values of a dtype is np.iinfo() or np.finfo().






In [0]:
print(np.iinfo(np.int64))
print(np.iinfo(np.int8))

We will **downcast** the one-hot columns and the windspeed to **int8** in the cell below with [pd.to_numeric()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.to_numeric.html). As an exercise, one might investigate wether an int64 is required for AirportID and optionally downcast it.

The to_numeric() method with the 'downcast' parameter will automatically pick the best possible dtype.

In [0]:
for columnname in ["WindSpeed","dir_0","dir_40","dir_80","dir_120","dir_160","dir_200","dir_240","dir_280","dir_320"]:
  weather[columnname] = pd.to_numeric(weather[columnname],downcast="integer")

**(2) floats**

The "float" type has subtypes float16, float32, float64 and float128. Currently, all floats are of dtype float64.

It is clear that the RelativeHumidity column has simple integer values disguised as floats. The decimal is always zero. We will downcast this to int8. 




In [0]:
# weather["RelativeHumididty"] = pd.to_numeric(weather["RelativeHumidity"], downcast="float")

The RelativeHumidity, Visiblity, StationPressure and Altimeter columns are floats with decimals, but there minimum and maximum values don't require a float64 - by far. Let's see the minimum and maximum values for some float types.

In [0]:
print(np.finfo(np.float64))
print(np.finfo(np.float32))
print(np.finfo(np.float16))


Looks like a smaller float will do. Downcast!

In [0]:
for columnname in ["StationPressure","Visibility","DryBulbCelsius","WetBulbCelsius","DewPointCelsius","RelativeHumidity"]:
  weather[columnname] = pd.to_numeric(weather[columnname],downcast="float")

In [0]:
weather.info(memory_usage="deep",max_cols=1)

In [0]:
weather.head()

# 3 Cleaning delay data



## 3.1 Dropping and one-hot-encoding

In [0]:
delays.head()

In [0]:
delays.dtypes

**DayOfWeek** has values ranging from 0 (Monday) to 6 (Sunday). We will not need this column so we drop it.

In [0]:
print("unique DayOfWeek: ",delays["DayOfWeek"].unique().shape[0])
delays["DayOfWeek"].unique()

In [0]:
delays.drop(columns = "DayOfWeek",inplace=True)

**Carrier** is a categorical value with 16 categories. 

In [0]:
print("unique Carrier: ",delays["Carrier"].unique().shape[0])
delays["Carrier"].unique()

We will one-hot encode this column. Below is a function to perform this one-hot-encoding. Notice the important memory optimization! Later we will use this function to one-hot encode the airportID's which creates over hundred new columns. You are not able to run this one-hot-encoding without the memory optimization line.

In [0]:
def one_hot(df,features):
    all_column_names = []
    for feature in features:
        names = list(df[feature].unique())
        for i, name in enumerate(names):
            column_name = str(feature)+'_'+str(i)
            df[column_name] = df[feature].apply(lambda x: 1 if x==name else 0)
            df[column_name] = pd.to_numeric(df[column_name],downcast='integer') #memory optimization!!
            all_column_names.append(column_name)
    df.drop(columns=features)
    return df

In [0]:
delays = one_hot(delays,["Carrier"])

## 3.2 Handling time

Departure and arrival times: we round **CRSDepTime** and **CRSArrTime** to the nearest hour, creating the columns **Dep_hour** and **Arr_hour**. We round *down*, while the hour of the weather was rounded *up*.  This is to make sure that weather conditions after the flight departure/arrival are not a deciding factor.

In [0]:
delays["Dep_hour"] = delays["CRSDepTime"].apply(lambda x: hhmm_to_h(x,mode="down"))
delays["Arr_hour"] = delays["CRSArrTime"].apply(lambda x: hhmm_to_h(x,mode="down"))

In [0]:
delays.head()

In [0]:
delays.columns

In [0]:
delays.dtypes

We will create columns **Dep_datetime** and **Arr_datetime** in the same format as the **Datetime** column in the weather dataset. We will use these columns later to join the delay data with the weather data.

By inspecting the columns above, we can see that the flight info has only one 'DayofMonth'. We assume this is the day of *departure*. However, when a flight takes place overnight, the arrival might be on the next day. This must be taken into account when constructing the Dep_datetime and Arr_datetime columns.

We assume the departure and arrival hour (Dep_hour and Arr_hour) are in the local timezone of the departure and arrival airports respectively. If departure time is later than arrival time, then the arrival is on the next day, so this must be taken into account when constructing Arr_datetime and Dep_datetime (add one day to Dep_datetime). 

We introduce the helper column **dep_after_arr** (1 or 0). This column is '1' if the arrival date in on the next day and '0' otherwise.



In [0]:
delays["dep_after_arr"] = (delays["CRSArrTime"]<delays["CRSDepTime"])

In [0]:
delays["Dep_datetime"] = delays["Year"].astype(str)+'-'+delays["Month"].astype(str)+'-'+delays["DayofMonth"].astype(str)+'-'+delays["Dep_hour"].astype(str)

We add an extra helper column **DayofMonth_arr**, the day of month of *arrival*. This is one day later than the **DayofMonth** column if arrival is one day later (i.e, when dep_after_arr is 1) and the same as **DayofMonth** otherwise.

In [0]:
delays["DayofMonth_arr"] = delays["DayofMonth"]+delays["dep_after_arr"]

From this, we can create **Arr_datetime**

In [0]:
delays["Arr_datetime"] = delays["Year"].astype(str)+'-'+delays["Month"].astype(str)+'-'+delays["DayofMonth_arr"].astype(str)+'-'+delays["Arr_hour"].astype(str)

## 3.3 Result

Let's drop some columns we no longer need.

In [0]:
delays.drop(columns=["dep_after_arr","DayofMonth","DayofMonth_arr","CRSArrTime", "CRSDepTime","Dep_hour","Arr_hour","Year","Month","Carrier"],inplace=True)

The result is as follows..

In [0]:
delays.head()

## 3.4 memory usage and dtypes



In [0]:
delays.info(memory_usage='deep')

In [0]:
delays.dtypes

In [0]:
delays.head()

DepDelay and ArrDelay are now floats but they can be casted to integers. ArrDelay15 and Cancelled are floats that are actually binary 1 or 0. The one-hot encoded features are int64, while an int8 should be sufficient. 

We will downcast all values as integers, except for the datetime columns.

In [0]:
delays.columns[0:-2]

In [0]:
for columnname in delays.columns[0:-2]: #excluse the last two (datetimes Dep_datetime and Arr_datetime objects)
  delays[columnname] = pd.to_numeric(delays[columnname],downcast="integer")

In [0]:
delays.dtypes

In [0]:
delays.info(memory_usage='deep')

# 4 Join with OriginAirport weather

Apparently, in the weather data sometimes for the same airport two measurements are available in the same hour (i.e. same datetime). We need to drop these duplicates (keeping the last available) so that we have unique airport-datetime combinations. Then, we can index on these columns (double index) and join the dataframes. 

In [0]:
weather.drop_duplicates(subset=["AirportID","datetime"],keep="last",inplace=True)

In [0]:
weather.set_index(keys=["AirportID","datetime"],inplace=True)

In [0]:
weather.head(5)

In [0]:
delays.dtypes

We are now ready to add the weather information to our delays dataframe. We will first join with the weather at the departure airport. 

In order to perform the join, we must temporarily rename our OriginAirportID to AirportID and Dep_datetime to datetime. The column names to be joined on, are then equal in both dataframes of the join.

In [0]:
delays.rename(columns={"OriginAirportID":"AirportID","Dep_datetime":"datetime"},inplace=True)

Now, adding the weather of the departure airport to the dataframe...

In [0]:
delays.head()

In [0]:
delays=delays.join(weather,on=["AirportID","datetime"])#,rsuffix="_depweather")

In [0]:
delays.columns

A lot of columns have been added related to the weather at the origin airport. We will rename all these columns with a suffix **_originairport** so that we can distinct them later from the weather data of the destination airport. We also change  AirportID back to OriginAirportID and datetime to Dep_datetime

In [0]:
delays_columns_old = delays.columns
delays_columns_new = []
for column in delays_columns_old:
    if column in weather.columns:
        delays_columns_new.append(column+"_originAirport")
    else:
        delays_columns_new.append(column)
        
delays.columns = delays_columns_new
delays.rename(columns={"AirportID":"OriginAirportID","datetime":"Dep_datetime"},inplace=True)
delays.columns

# 5 Join with DestinationAirport weather

We now perform the same tricks with the destination airport weather.

In [0]:
delays.rename(columns={"DestAirportID":"AirportID","Arr_datetime":"datetime"},inplace=True)

In [0]:
delays=delays.join(weather,on=["AirportID","datetime"])#,rsuffix="_arrweather")

In [0]:
delays.columns

In [0]:
delays_columns_old = delays.columns
delays_columns_new = []
for column in delays_columns_old:
    if column in weather.columns:
        delays_columns_new.append(column+"_destAirport")
    else:
        delays_columns_new.append(column)
        
delays.columns = delays_columns_new
delays.rename(columns={"AirportID":"DestAirportID","datetime":"Arr_datetime"},inplace=True)
delays.columns

In [0]:
delays.head()

In [0]:
delays.dtypes

# 6 Final data processing and result

Finally, we drop some unneeded columns. Let's assume that the exact flight times don't matter, only the weather conditions at those times. For now the only target variable is ArrDelay15, we exclude other related columns.

In [0]:
delays.drop(columns=["Dep_datetime","Arr_datetime","DepDelay","DepDel15","ArrDelay","Cancelled"],inplace=True)


Dropping NaN values

In [0]:
delays.dropna(inplace=True)

In [0]:
delays.dtypes

During the join, our columns related to Windspeed and Wind Direction are now back to float64. Cast them back to integers to conserve memory

In [0]:
delays.info(memory_usage='deep',max_cols=1)

In [0]:
def isDowncastColumn(columnName):
    if ("dir_" in columnName) or ("WindSpeed" in columnName):
      return True
    else:
      return False
    
for columnName in filter(isDowncastColumn,list(delays.columns)):
  delays[columnName] = pd.to_numeric(delays[columnName],downcast='integer')

In [0]:
delays.dtypes

In [0]:
delays.info(memory_usage='deep',max_cols=1)

Now, we must one-hot-encode the destination and origin airport ID's. Since there are 64 options for each, this will introduce 128 new one-hot features!

In [0]:
print("unique origin airports: {0}".format(delays["OriginAirportID"].unique().shape[0]))
print("unique destination airports: {0}".format(delays["DestAirportID"].unique().shape[0]))

In [0]:
#this might take a few minutes...
delays = one_hot(delays,["OriginAirportID","DestAirportID"])

Our resulting dataset has 61 columns (60 feature columns and one target output). Our data preprocessing was limited to one-hot-encoding, cleaning the data, and doing some manipulations on the time information. No advanced feature engineering was done.

In [0]:
delays.head(5)

In [0]:
delays.info(memory_usage='deep')

In [0]:
delays.columns

Our target variable is **ArrDel15**, a binary (0 or 1) indicating wether or not the flight arrives at least 15 minutes late.

In [0]:
delays["ArrDel15"].head()

In [0]:
# saving the dataframe to a CSV file. We will use it later.

delays.to_csv("/content/drive/My Drive/xylosai/flightDelay/FlightDelaysData_clean.csv")

# 7 Logistic regression

## 7.1 Introduction

In this notebook we will build a binary classifier using logistic regression. Our goal is to build a model that is able to determine wether or not a commercial aircraft flight will be delayed or not, based on information such as the weather conditions and the carrier of the flight.

In logistic regression for binary classification, there are only 2 classes: 1 and 0. An output Z is calculated as a linear function of the features. The result of this is "activated" with a softmax activation function, causing the resulting output to fall in the interval ]0,1[, interpreted as a probability P(Y'=1). Then, at prediction time, we will predict "1" if the output is above some threshold (usually 0.5).

$Z = w_0 + w_1h_1 + w_2h_2 + ... + w_nh_n$

$Y' =  sigmoid(Z) = 1/(1+exp(-Z)$


Logistic regression is a **linear classifier**. It can only find a linear decision boundary in the features $h_i$. In general, features are any function of the "raw" inputs $x_i$. This allows to learn a non-linear relationship between output and input. In our case, the features are all the columns in the dataset (except for the target column).





## 7.2 The implications of linearity

The implications of linearity can be explained better with the **WindDirection** input. It is reasonable to assume that the wind direction has an influence on flight delays. Maybe, if the wind direction is perpendicular to the direction of take-off, take-off might be impeded. 

The possible values for the wind direction range from 0° to 360° plus some 'VR' case. Going back to our formula, if one of the features was the winddirection, a weight must be learned that encodes the influence of wind direction. 

Learning a positive weight would mean that a "high" degree (closer to 360°) causes flight delay. Then, $Z$ will be higher so that the output $Y'$ is close to one, leading to a prediction of 1 (1 = delay, 0 = no delay). 

Generally, there is no reason to believe that a "high" degree (close to 360°) is better for take-off then a "low degree" (close to 0°). With feature engineering, we could find a non-linear relationship with wind direction, but then we still have no way to deal with the 'VR' case.

But we have one-hot-encoded our wind direction in ranges of 40°. Each of these one-hot-encoded features has its own weight. If the wind direction is 165°, only the weight of the one-hot-encoded feature **dir_160** will influence the outcome, since all other one-hot-encoded features are zero. This way, the model can learn that, for example, a wind direction of 160° causes delay (a high weight) and wind directions close to 0° or close to 360° are less likely to cause delay (a low weight).

This could work if there was only airport in the dataset. But our model - and its weights - applies to all airports in the dataset. Different airports have a different orientation. A wind direction of 165° degree might be beneficial for one airport, but not for the other. Therefore, it is unlikely to assume that weights can be learned that work for all airports. In order to truly take into account the wind direction, some single feature $h_i$ should be engineered that is a function of both airportID and wind direction simultaneously. It is unclear how this feature should look. **It looks like taking into account wind direction is not easily done with a linear classifier like logistic regression...**. For logistic regression, we will drop features related to wind direction (**dir_0** to **dir_320**.)

We will later see that the problem of manual feature engineering is addressed with neural networks. Neural networks are able to find non-linear relationships between input and output, without feature engineering. With neural networks, wind direction would not even have to be one-hot-encoded if consisted of only numerical value and it did not contain the weird 'VR' value. Unfortunately, because of this 'VR' value, we will keep the one-hot-encoded wind direction features even for the neural network example later.



## 7.3 Building a first model

Drop columns related to wind direction.

In [0]:
#might be needed to conserve memory
del weather

In [0]:
from sklearn.linear_model import LogisticRegression

In [0]:
def isDirectionColumn(columnName):
    if "dir_" in columnName:
      return True
    else:
      return False
    


In [0]:
directionColumns = filter(isDirectionColumn,list(delays.columns))

In [0]:
delays = delays.drop(columns = directionColumns) #note: inplace = False (default)


In [0]:
delays.head()

In [0]:
delays.columns

In [0]:
Y = delays["ArrDel15"].astype(np.float64)
X = delays.drop(columns=["ArrDel15"]).astype(np.float64)

In [0]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,random_state=0,test_size = 0.10)

Creating a model. Read the [documentation](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) for logistic regression in sklearn. Are we applying any regularization?

In [0]:
model_logreg = LogisticRegression(random_state = 0)
model_logreg.fit(X_train,Y_train)

## 7.4 Evaluating the model

the score() method returns the accuracy of a model on a dataset. As usual, we evaluate our model on the unseen **test set**

In [0]:
accuracy = model_logreg.score(X_test,Y_test)
print("Accuracy: {0}".format(accuracy))

In [0]:
perc_ontime = float(len(delays[delays["ArrDel15"] == 0]))/len(delays)
print("fraction of flights that are on time: {0}".format(perc_ontime))

The accuracy of 79% is not a good performance indicator. Since 79% percent of the dataset is not-delayed (ArrDel15 = 0), a majority class classifier would achieve the same accuracy. Maybe the model has simply learned that basically all flights are on time?

In [0]:
predictions = model_logreg.predict(X_test)
pred_positive = np.sum(predictions == 1)
pred_negative = np.sum(predictions == 0)
print("predicted positive: {0}".format(pred_positive))
print("predicted negative: {0}".format(pred_negative))




We calculate the Area Under the (ROC) Curve (AUC) using Sklearn's builtin functionality ([documentation](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression.score)). First we must predict the probabilities without applying a threshold with [predict_proba](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression.predict_proba). Predict_proba returns a Numpy array with the probabilities of each class, for each data point. The sum of each row (the sum of both columns in this case of binary classification) is one. 

In [0]:
Y_test_prob = model_logreg.predict_proba(X_test)
Y_test_prob

Our function for calculating AUC requires the probability estimates of the positive class, i.e. the second column of Y_test_prob (since columns are ordered by the label of classes, so the first column is class 0 and class 1).

In [0]:
auc = sklearn.metrics.roc_auc_score(Y_test,Y_test_prob[:,1])
print(auc)

We can also plot the roc curve of a binary classifier with the [roc_curve](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_curve.html) method and matplotlib. A **good classifier has**, for all thresholds, a **high true positive rate** while still maintaining a **low false positive rate**, i.e. a high area under the curve. A worthless random classifier (baseline) has AUC = 0.5 and is drawn as a 45° degree straight line: fpr and tpr are equal for any threshold. Let's plot our ROC curve together with the baseline. We create a function plotROC for this, since we will need to do this again later.

In [0]:
fpr, tpr, tresholds = sklearn.metrics.roc_curve(Y_test, Y_test_prob[:,1])
 

In [0]:
def plotROC(fpr, tpr):
  fig = plt.figure(figsize = (10,10))
  plt.xlabel("false positive rate (FPR)",fontsize = 15)
  plt.ylabel("true positive rate (TPR)",fontsize = 15)
  plt.title("ROC curve",fontsize=20)
  plt.plot(fpr, tpr,"b",fpr, fpr, "k:")
  plt.legend(("ROC curve","baseline"),fontsize=15)
  plt.show()

  

In [0]:
plotROC(fpr, tpr)

## 7.5 Dealing with imbalanced classes (undersampling)

In our data, 79% of flights were on time (class 0). Therefore, our model is biased towards predicting 0. We will now balance our training data so that it contains an equal amount of positive (1) and negative (0) samples. With this balanced set we fit a new classifier, and evaluate on the same test set.

Note that we balance our training set, but we still evaluate our model on the original test set so that the test

In [0]:
# category count before balancing
print("not delayed: {0}".format(np.sum(Y_train == 0)))
print("delayed: {0}".format(np.sum(Y_train == 1)))

In [0]:
# THIS IS THE ORIGINAL CODE FOR BALANCING THE CLASSES. READ AND UNDERSTAND IT FIRST.
# THE CELL BELOW PERFORMS THE SAME TASK, BUT IS MORE MEMORY-OPTIMIZED. 
# (memory is limited to about 12GB in Colaboratory).

# #split training data into classes
# X_train_1 = X_train[Y_train == 1]  #minority
# Y_train_1 = Y_train[Y_train == 1]

# X_train_0 = X_train[Y_train == 0]  #majority
# Y_train_0 = Y_train[Y_train == 0]

# X_train_0 = X_train_0[0:len(Y_train_1)]
# Y_train_0 = Y_train_0[0:len(Y_train_1)]

# #shapes should be equal
# print(X_train_1.shape, X_train_0.shape)
# print(Y_train_1.shape, Y_train_0.shape)

# X_train_balanced = pd.concat([X_train_1, X_train_0],axis=0)
# Y_train_balanced = pd.concat([Y_train_1, Y_train_0],axis=0)

# #shuffling first could improve training
# X_train_balanced, Y_train_balanced = sklearn.utils.shuffle(X_train_balanced, Y_train_balanced, random_state=0)

In [0]:
# balancing the classes in the train set, memory-optimized

l_1 = np.sum(Y_train == 1)


X_train = pd.concat([X_train[Y_train == 1], X_train[Y_train == 0][0:l_1]],axis=0)
Y_train = pd.concat([Y_train[Y_train == 1], Y_train[Y_train == 0][0:l_1]],axis=0)

X_train, Y_train = sklearn.utils.shuffle(X_train, Y_train, random_state=0)

In [0]:
# category count after balancing
print("not delayed: {0}".format(np.sum(Y_train == 0)))
print("delayed: {0}".format(np.sum(Y_train == 1)))

Training a new model on balanced training set, and evaluate on the (non-balanced) test set.

In [0]:
model_logreg = LogisticRegression(random_state = 0)
model_logreg.fit(X_train,Y_train)

In [0]:
Y_test_prob = model_logreg.predict_proba(X_test)

In [0]:
auc = sklearn.metrics.roc_auc_score(Y_test,Y_test_prob[:,1])
auc

With undersampling, our AUC score has improved slightly from 0.61 to 0.66.

It is important to mention that for a different random subset of the training data, or a different random test-train split, the results can be slightly different.

In [0]:
fpr, tpr, tresholds = sklearn.metrics.roc_curve(Y_test, Y_test_prob[:,1])

In [0]:
plotROC(fpr, tpr)

# X Dense neural network (to do)

In [0]:
from keras import layers
from keras.layers import Input, Add, Dense, Activation, Dropout, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.initializers import glorot_uniform
import tensorflow as tf

In [0]:
def delayNNmodel(input_shape,hidden_units = [100,100],dropout_rate = 0.1):
    
    #input layer
    X_input = Input(input_shape)
    
    #first hidden layer
    X = Dense(hidden_units[0])(X_input)
    X = Activation('relu')(X)
    X = Dropout(dropout_rate,seed=0)(X)
    
    #next hidden layer(s)
    if len(hidden_units)>2:
        for n in hidden_units[1:]:
            X = Dense(n)(X)
            X = Activation('relu')(X)
            X = Dropout(dropout_rate,seed=0)(X)
    elif len(hidden_units) == 2:
        X = Dense(hidden_units[1])(X)
        X = Activation('relu')(X)
        X = Dropout(dropout_rate,seed=0)(X)
        
    #output layer
    X = Dense(1)(X)
    X = Activation('sigmoid')(X)
    
    model = Model(inputs = X_input, outputs = X, name = 'delayBinaryClassifierNN')
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [0]:
def evaluate_NN(model,X,Y,plot=False):
    Y_pred = model.predict(X)
    fpr,tpr,thresholds = roc_curve(Y,Y_pred)
    if plot:
        plt.plot(fpr,tpr)
    AUC = auc(fpr,tpr)
    return AUC

In [0]:
def fit_and_plot(model,X_train,Y_train,X_test,Y_test,epochs = 10,verbose=False,evaluate_offset = 0):
    auc_test = []
    auc_train = []
    for i in range(epochs):
        print(i)
        model.fit(X_train,Y_train,epochs=1,batch_size=128,verbose=0)
        if i >= evaluate_offset:
            auc_test_ = evaluate_NN(model,X_test,Y_test,plot=False)
            auc_test.append(auc_test_)
            auc_train_ = evaluate_NN(model,X_train,Y_train,plot=False)
            auc_train.append(auc_train_)
            if verbose:
                print("Epoch {2}. Test auc: {0}, Train auc: {1}".format(auc_test_,auc_train_,i))
    plt.figure()
    plt.plot(np.arange(len(auc_test)),auc_test,'r',np.arange(len(auc_train)),auc_train,'b')
    plt.show()
    return model

**never name the output of this function 'auc' since it collides with the name of the function auc, imported above**

auc_ = evaluate_NN(model,X_test,Y_test,plot=False)
print(auc_)

**Output is always 0, see cell below. Let's try to balance to dataset first, since it is skewed towards output 0**

In [0]:
pred=model.predict(X_train)
np.sum(pred>0.00001)

In [0]:
np.sum(Y_train==1)/Y_train.shape[0]

In [0]:
np.sum(Y_train==0)/Y_train.shape[0]

In [0]:
#turn this into a robust reusable function!

index_0 = (Y_train==0)
index_1 = (Y_train==1)
print(np.sum(index_0))
print(np.sum(index_1))
amt_1 = np.sum(index_1)
Y_train_0 = Y_train[index_0]
Y_train_1 = Y_train[index_1]
X_train_0 = X_train[index_0]
X_train_1= X_train[index_1]

np.random.seed(0)
selection_0 = np.random.randint(0,Y_train_0.shape[0],amt_1)

Y_train_0_selected = Y_train_0[selection_0]
X_train_0_selected = X_train_0[selection_0]

X_train_balanced = np.append(X_train_0_selected,X_train_1,axis=0)
Y_train_balanced = np.append(Y_train_0_selected,Y_train_1,axis=0)

print(X_train_balanced.shape)
print(np.expand_dims(Y_train_balanced,axis=1).shape)

XY = np.append(X_train_balanced,np.expand_dims(Y_train_balanced,axis=1),axis=1)
np.random.shuffle(XY)
X_train_balanced = XY[:,0:-1]
Y_train_balanced = XY[:,-1]

print(X_train_balanced.shape)
print(Y_train_balanced.shape)

In [0]:
input_shape = [X_train.shape[1]]
model = delayNNmodel(input_shape,dropout_rate=0)
fit_and_plot(model,X_train_balanced,Y_train_balanced,X_test,Y_test,verbose=True,epochs=3)

In [0]:
auc_ = evaluate_NN(model,X_test,Y_test,plot=False)
print(auc_)

In [0]:
units = [[128,128],[128,128,128],[256,256,128]]
dropout_rates = [0,0.05,0.1,0.2]
for u in units:
    for dropout_rate in dropout_rates:
        model = delayNNmodel(input_shape, hidden_units = u, dropout_rate = dropout_rate)
        print("units: {0}, dropout: {1}".format(u,dropout_rate))
        fit_and_plot(model,X_train_balanced,Y_train_balanced,X_test,Y_test,verbose=False,epochs=6)

**doing extra epochs for those who seem promising**

In [0]:
units = [[128,128],[128,128,128],[128,128,128],[256,256,128]]
dropout_rates = [0,0,0.05,0.05]
for u,dropout_rate in zip(units,dropout_rates):
        model = delayNNmodel(input_shape, hidden_units = u, dropout_rate = dropout_rate)
        print("units: {0}, dropout: {1}".format(u,dropout_rate))
        fit_and_plot(model,X_train_balanced,Y_train_balanced,X_test,Y_test,verbose=False,epochs=10)

In [0]:
units = [[128,128,128],[256,256,128]]
dropout_rates = [0,0.05]
for u,dropout_rate in zip(units,dropout_rates):
        model = delayNNmodel(input_shape, hidden_units = u, dropout_rate = dropout_rate)
        print("units: {0}, dropout: {1}".format(u,dropout_rate))
        fit_and_plot(model,X_train_balanced,Y_train_balanced,X_test,Y_test,verbose=False,epochs=20)

In [0]:
units = [[256,256,128]]
dropout_rates = [0.05]
for u,dropout_rate in zip(units,dropout_rates):
        model = delayNNmodel(input_shape, hidden_units = u, dropout_rate = dropout_rate)
        print("units: {0}, dropout: {1}".format(u,dropout_rate))
        fit_and_plot(model,X_train_balanced,Y_train_balanced,X_test,Y_test,verbose=False,epochs=40)

In [0]:
units = [[256,256,128]]
input_shape = [X_train.shape[1]]
dropout_rates = [0.05]
for u,dropout_rate in zip(units,dropout_rates):
        model = delayNNmodel(input_shape, hidden_units = u, dropout_rate = dropout_rate)
        print("units: {0}, dropout: {1}".format(u,dropout_rate))
        fitted_model = fit_and_plot(model,X_train_balanced,Y_train_balanced,X_test,Y_test,verbose=False,epochs=80,
                                    evaluate_offset=30)